In [33]:
# Imports
import os 
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = 'true'

In [34]:
from django.forms import model_to_dict
from flourish_child.models import TbReferalAdol as TbReferalAdolCrf
from edc_metadata.constants import REQUIRED, KEYED
from edc_appointment.constants import COMPLETE_APPT
from django.db import IntegrityError
    
visit_code = '2100A'

tb_referral_objs = TbReferalAdol.objects.all()

for obj in tb_referral_objs:
    instance = model_to_dict(obj)
    
    del instance['action_identifier']
    del instance['parent_tracking_identifier']
    del instance['related_tracking_identifier']
    del instance['subject_identifier']
    del instance['tracking_identifier']
    
    try:
        child_visit = ChildVisit.objects.get(visit_code=visit_code,
                                             visit_code_sequence = 0,
                                             subject_identifier = obj.subject_identifier)
        
    except ChildVisit.DoesNotExist:
        pass
    else:
        crf = TbReferalAdolCrf(
            child_visit=child_visit,
            **instance
        )
        
        try:
            crf.save()
            print(f'{child_visit.subject_identifier} : Created')
        except IntegrityError:
            print(f'{child_visit.subject_identifier} : Already Created')
        
        # Delete metadata auto create
        CrfMetadata.objects.filter(
            subject_identifier = child_visit.subject_identifier,
            visit_code = child_visit.visit_code,
            visit_code_sequence = child_visit.visit_code_sequence,
        ).delete()
        
        # Update the metadata
        child_visit.save()
        

    



B142-040990838-0-10 : Already Created
B142-040990705-1-10 : Already Created
B142-040991059-2-10 : Already Created
